# Curve interpolation resampling(曲线插值)

This method is used for interpolation of non ideal resolution curves and provides multiple interpolation methods该方法用于对非理想分辨率的曲线进行插值，提供多种插值方法

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import math
import seaborn as sns
import pylab as pl

from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import interpolate
from pandas import set_option
set_option("display.max_rows", 15)

In [2]:
# 使用GPU训练时候，打开下面注释
import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:
# 定义要处理的常规测井资料
usual_log_names = ["AC","CNL","DEN","GR","RD","RS"]
# usual_log_names = ["DT","DTXX"]
# usual_log_names = ["DTC","DTS"]

# Set the resolution for sampling（设定新的分辨率）

In [4]:
# 需要提升的分辨率，为插值间点数:
# 如果间隔0.1m, aim_resolution = 10； 
# 间隔0.125m，aim_resolution = 8；
# 间隔1m，aim_resolution = 1；以此类推
# # 确保目标曲线的分辨率能够和测井的相匹配以制作标签
aim_resolution = 8

In [5]:
# 元素单位表示单位是否是百分比 default False（测井） | True(录井) 
use_percent = False

# Load csv data（读入原始数据）

In [6]:
# 定义是否要使用常规测井资料，True | False
use_usual_log = True

In [7]:
if use_usual_log == True:
    data_path = 'data/usual_logs/'

In [8]:
# filename 取值例如：'TT1-4500m-5070m_R_0.1524.csv','HP1_4075m-5582m_R_1.csv'
filename = 'GY1_R_0.1524m_整段_1700m-2300m.csv'
element_logs_file = os.path.join(data_path,filename)
# 读取A、B部分共有数据
un_process = pd.read_csv(element_logs_file,engine='python',encoding='GBK')

In [9]:
# 删除NAN行
un_process_YS = un_process.dropna()
un_process_YS

,DEPTH,DTC,DTS
0,5617.0753,75.3691,155.1939
1,5617.2277,74.9615,153.6787
2,5617.3801,74.5912,151.0225
3,5617.5325,73.8546,147.7337
4,5617.6849,72.8347,145.1820
5,5617.8373,72.5753,144.4313
6,5617.9897,73.4508,145.5181
...,...,...,...
3111,6091.1917,85.1627,169.4444
3112,6091.3441,85.5282,169.5480


In [10]:
# 删除重复行
un_process_YS = un_process_YS.drop_duplicates(subset='DEPTH', keep='last')
# un_process_YS.set_index('DEPTH')
print(un_process_YS.shape)

(3118, 3)


# Determine interpolation range（确定插值范围）

In [11]:
C_DEPTH = un_process_YS.loc[:,["DEPTH"]]
C_DEPTH

,DEPTH
0,5617.0753
1,5617.2277
2,5617.3801
3,5617.5325
4,5617.6849
5,5617.8373
6,5617.9897
...,...
3111,6091.1917
3112,6091.3441


In [12]:
X = np.array(C_DEPTH)
X.shape = (len(X),)

In [13]:
X[0] = math.floor(X[0])
X[0]

5617.0

In [14]:
X[-1] = math.floor(X[-1])
X[-1]

6092.0

In [15]:
if use_usual_log == True:
    C_element = un_process_YS.loc[:, usual_log_names]
C_element

,DTC,DTS
0,75.3691,155.1939
1,74.9615,153.6787
2,74.5912,151.0225
3,73.8546,147.7337
4,72.8347,145.1820
5,72.5753,144.4313
6,73.4508,145.5181
...,...,...
3111,85.1627,169.4444
3112,85.5282,169.5480


In [16]:
## math.ceil(f) #向上取整, math.floor(f) #向下取整
## X_min、X_max可以由用户指定，也可以是当前测井日志中的最大范围
# X_min and X_max can be specified by the user, or they can be the maximum range in the current logging log
X_min = math.floor(np.min(X))
X_max = math.floor(np.max(X))
print(X_min, X_max)

5617 6092


In [17]:
# Xnew = np.linspace(X_min, X_max,(X_max - X_min)* resolution + 1)
# Xnew = np.linspace(np.min(X),np.max(X),(np.max(X)-np.min(X))* resolution + 1)
# 上面的代码由于numpy版本1.问题会报错，https://stackoverflow.com/questions/59801932/raise-typeerror-typeerror-object-of-type-class-numpy-float64-cannot-be-saf

Xnew = np.linspace(np.min(X),np.max(X),(X_max-X_min)* aim_resolution + 1)
# Xnew

In [18]:
# print("Xnew.shape,type(Xnew):", Xnew.shape,type(Xnew))
if (len(Xnew) -((X_max-X_min)* aim_resolution + 1)) == 0:
    print("x axis is right!")

x axis is right!


# Interpolate and label the curves of each element based on resolution requirements(针对分辨率要求对各元素曲线进行插值制作标签)

In [19]:
pd_data0 = pd.DataFrame(Xnew,columns=["DEPTH"])
pd_data = pd_data0  
if use_usual_log == True:
    object_names = usual_log_names

for i in range(len(object_names)):
    element_Y = []
    element_Y = C_element.loc[:, object_names[i]]
    if use_percent == True:
        element_Y = element_Y / 100.
    Y = np.array(element_Y)
    Y.shape = (len(Y),)
    Ynew = np.zeros(len(Xnew))

    # for kind in ["nearest","zero","slinear","quadratic","cubic"]:#插值方式
    #     #"nearest","zero"为阶梯插值
    #     #slinear 线性插值
    #     #"quadratic","cubic" 为2阶、3阶B样条曲线插值
    #     f=interpolate.interp1d(X,Y,kind=kind)
    #     # ‘slinear’, ‘quadratic’ and ‘cubic’ refer to a spline interpolation of first, second or third order)
    #     Ynew=f(Xnew)
    #     pl.plot(Xnew,Ynew,label=str(kind))
#     print(Y.shape,type(Y))
    method_kind = "slinear"
    f = interpolate.interp1d(X, Y, kind = method_kind)
        # ‘slinear’, ‘quadratic’ and ‘cubic’ refer to a spline interpolation of first, second or third order)
    Ynew = f(Xnew)
#     results.append(Ynew)

    pd_data1 = pd.DataFrame(Ynew,columns = [object_names[i]])

    pd_data =pd.concat([pd_data,pd_data1],axis=1)
#     pd_data1 = pd.DataFrame(Ynew,columns=[element_name[i] +"_chazhi"])
#     pd_data = pd.concat([pd_data0,pd_data1],axis=1)

In [20]:
pd_data

,DEPTH,DTC,DTS
0,5617.000,75.369100,155.193900
1,5617.125,75.145341,154.362104
2,5617.250,74.907316,153.290030
3,5617.375,74.603592,151.111389
4,5617.500,74.011683,148.435052
5,5617.625,73.235566,146.184932
6,5617.750,72.723893,144.861327
...,...,...,...
3794,6091.250,85.302521,169.484032
3795,6091.375,85.811632,170.664273


# Processing interpolated data(对插值后的数据处理)

In [21]:
# 数据小于0的变为0
for i in range(len(object_names)):
    element_value =  pd_data.iloc[:,(i + 1)]
    for j in range(len(element_value)):
        if element_value[j] < 0:
            element_value[j] = 0

# Save data to csv(结果写入csv)

In [22]:
import datetime
def tid_maker():
    return '{0:%Y%m%d%H%M}'.format(datetime.datetime.now())

In [23]:
csvfile_name = (filename.split(".csv")[0]).split("_R_")[0] #'TT1-4500m-5070m_R_0.1524.csv'
print(csvfile_name)

高102


In [24]:
print(str(1/aim_resolution))

0.125


In [25]:
start_depth = X_min
end_depth = X_max

In [26]:
pd_data.to_csv(data_path + csvfile_name + '_new_R_'+ str(1/aim_resolution) + '_'+ str(start_depth) + 'm-'+str(end_depth)+ "m" + '.csv',mode='w',index = False,header=True)

In [ ]:
print("Correcting well logs is finished!!")
print(data_path + csvfile_name + '_new_R_'+ str(1/aim_resolution) + '_'+ str(start_depth) + 'm-'+str(end_depth)+ "m" + '.csv')